# 📦 Week 09-10 · Notebook 06 · Mini-batch Training & Curriculum Scheduling

Design batching strategies and curricula to stabilize training on skewed manufacturing corpora.

## 🎯 Learning Objectives
- Implement streaming data loaders for mixed PDF/CSV corpora.
- Compare curriculum strategies for rare incident coverage.
- Balance batch size vs. throughput vs. GPU memory.
- Build shift hand-off SOPs for 24/7 training operations.

## 🧩 Scenario
Only 3% of maintenance logs capture critical failures, yet they matter most. You need a training curriculum that ramps from routine notes → cautionary signals → critical incidents.

In [ ]:
import math
import random
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

torch.manual_seed(909)

## 🗂️ Curriculum Tags
Each sample is assigned a severity level. Critical incidents are oversampled later in the curriculum.

In [ ]:
def create_curriculum_dataset(n=1200):
    severities = ['routine', 'warning', 'critical']
    severity_weights = [0.82, 0.15, 0.03]
    records = []
    for i in range(n):
        level = random.choices(severities, weights=severity_weights)[0]
        embedding = torch.randn(512)
        if level == 'critical':
            embedding += 2.0
        elif level == 'warning':
            embedding += 0.5
        records.append({'id': i, 'severity': level, 'embedding': embedding})
    return records

curriculum_records = create_curriculum_dataset()
pd.Series([r['severity'] for r in curriculum_records]).value_counts(normalize=True)

In [ ]:
class CurriculumDataset(Dataset):
    def __init__(self, records, severity_priority):
        self.records = records
        self.severity_priority = severity_priority

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        record = self.records[idx]
        return record['embedding'], self.severity_priority[record['severity']]

severity_priority = {'routine': 0, 'warning': 1, 'critical': 2}
dataset = CurriculumDataset(curriculum_records, severity_priority)

## 🗜️ Adaptive Batch Sampler
Start with routine samples, then gradually mix in higher-severity items.

In [ ]:
def curriculum_indices(epoch, severity_priority, dataset):
    base = [i for i, _ in enumerate(dataset.records) if severity_priority[dataset.records[i]['severity']] == 0]
    warning = [i for i, _ in enumerate(dataset.records) if severity_priority[dataset.records[i]['severity']] == 1]
    critical = [i for i, _ in enumerate(dataset.records) if severity_priority[dataset.records[i]['severity']] == 2]
    schedule = {
        0: base,
1
        2: base + warning + critical
,
,
,
,
,
,
,